# 累積和とその周辺

# はじめに

この文書では、累積和、及びその周辺のアルゴリズム・データ構造について解説を行う。

累積和は、その内容自体は非常に簡単なものだが、非常に様々な問題においてあちらこちらに顔を出すので、重要なものでもある。

まずは累積和の基本的な考え方、実際の構築の仕方と利用方法から話を始め、ちょうど累積和の逆ともいえる通称「いもす法」、座標圧縮との組み合わせ、二次元累積和などに話を広げていく予定となっている。

# 累積和の基本

## なぜ累積和を使うのか？

ある $N$ 個の整数の要素を持つ数列 $A: A_1, A_2, ... , A_N$ があるとする。例えば、具体的には次のようなものだ。

$
2, -5, 42, 9, -38, 24, 19, -57, 88, 67
$

こういった感じの数列中の、連続した一部の合計、つまり連続部分和を求めることを考える。具体的に、```sum_part()``` の形で関数を作成することにしよう。ややこしくならないように、0-indexedの半開放で定義しておく。すなわち、```sum_part(a, b)``` のとき、返ってくる値は部分和 $A_{a-1} + A_{a} + ... + A_{b-2} + A_{b-2}$ とする。

また、あらかじめ $A$ はリストとして ```A``` に保存されているものとする。

In [ ]:
A = [2, -5, 42, 9, -38, 24, 19, -57, 88, 67]

さて、単純に要素を足し合わせるだけなので、もちろん次のように書ける。

In [ ]:
def sum_part(a, b):
    return sum(A[a:b])

print(sum_part(3, 6))  # = 9 - 38 + 24

一回こっきり部分和を計算するだけならば、この方法でなんの問題もない。ただ、何度もおなじ計算を要求されるようなケースでは、関数ごとに計算量が $O(N)$ となるこの方法では、問題がある。

例えば、回答として $A_1, A_1+A_2, A_1+A_2+A_3 ... $ を全て求めよ、などと言われた場合はどうだろうか？上記のやり方だと計算量は $O(N^2)$ となり、$N \le 10^4$ あたりから間に合わなくなってくる。

……いや、でも、その場合は前から順番に足していって、その都度答えを出力すればいいだけじゃない？と思ったことだろう。  
まさに、それこそが累積和である。

## 累積和を求める

というわけで、さっそく累積和を使ってみよう。

累積和自体は広い概念を指す言葉だが、競技プログラミングにおいて累積和を用いるというとき、多くの場合は、連続部分和を計算するために事前計算を行うことを指す。つまり、最初に次のようなものを計算する。

$
[0, A_1, A_1+A_2, A_1+A_2+A_3, ... , \sum_{i=1}^{N}A_i]
$

最初に 0 を入れているのは、後で都合がいいからだ。さっそく計算させてみよう。

In [ ]:
N = len(A)
cum_sum = [0] * (N+1)  # 累積和を保存していく

tmp = 0  # 現在の累積和を管理

for i in range(N):
    tmp += A[i]
    cum_sum[i+1] = tmp

print(cum_sum)

ちなみに、実は Python には累積和を自動的に求めてくれる ```itertools``` ライブラリの関数 ```accumulate``` が存在している。使用するためには、もちろんインポートする必要がある。

```itertools.accumulate``` は、キーワード引数 ```func``` で行う計算を柔軟に変更したり、```initial``` で初期値を設定したりもできる便利な関数だ。さっきの例なら、こう書ける。

In [ ]:
from itertools import accumulate
N = len(A)
cum_sum = accumulate(A, initial=0)
print(cum_sum)
print(list(cum_sum))

見ての通り返ってくるのはイテレータなので、添字を利用するならリストに変換する必要がある。

こちらを使用するほうが便利なのだが、個人的には自分で書くことが多い。

さて、いずれにせよ、こうして得られた数列がなぜ便利なのかというと、連続部分和を素早く計算できるからだ。

例えば、$A_3+A_4+A_5$ を計算したいとする。これは、$A_1+A_2+A_3+A_4+A_5$ から、$A_1+A_2$ を引いたものに等しくなる。この計算を一般化することにより、あらゆる連続部分和を、二つの部分和の差から簡単に計算できるようになる。

というわけで、最初に書いた ```sum_part()``` を書き直してみよう。

In [ ]:
csum_A = [0] * (N+1)
tmp = 0
for i in range(N):
    tmp += A[i]
    csum_A[i+1] = tmp

def sum_part(a, b):
    return csum_A[b] - csum_A[a]

sum_part(3, 6)

これで、事前準備 $O(N)$ により、連続部分和を $O(1)$ で計算できるようになったのが分かると思う。

# いもす法

## いもす法とは

いもす法とは、ある一次元以上の空間における範囲操作を差分のみで簡便にあらわしていく手法で、累積和とは微分・積分のような関係に近い（と、個人的には思っている）。

その利点は、範囲に対する操作を $O(1)$ で行える点だが、その情報の取り出しには $O(N)$ を要する。

つまり、操作のクエリが大量にあり、最終的な状態を一回答えればいいような問題に有効な手法といえる。


いささか抽象的な説明だったが、少し具体的な例を考えてみよう。  
ある駐車場があり、各車の入庫と出庫の時刻が記録されているとする。駐車場が混雑していた、仮に10台以上が駐車していた時間はどれくらいの長さになるのかを知りたいとしたとき、どのようにすればいいだろうか？ただし、出庫時刻には車は駐車していないものとする。

まずは、敢えて単純な解き方から考えてみる。

時間軸を取るので、ひとまず ```parking``` というリストを作成することにしよう。```parking[t]``` は、時刻 $t$ に駐車場に止まっている車の台数とする。

各車ごとに入出庫の時間は分かっている。それぞれを $T_1, T_2$ とする。出庫の時刻にはいなくなっているので、$T_1$ ～ $T_2-1$ の時刻に対し、駐車場に止まっている台数に 1 を加えることになる。コードに起こすと、次のような感じだろうか。

In [ ]:
for t in range(t1, t2):
    parking[t] += 1

このコードの場合、問題となるのは、車 1台の情報を処理するのに $O(max(t))$ かかってしまうことだ。車の台数が少なければ問題ないが、多くなってくると間に合わなくなってしまう。

よって、解き方を変える必要がある。

ところで、我々が現実に時間軸に沿って駐車台数を確認していくときはどうするだろうか？いちいち毎時刻ごとに車を数え直したりはしないだろう（もちろん、時々は確認するだろうが）。  
その代わりに、さっき 3台駐車していたところに、2台入庫してきて、1台出庫したから現在は 4台……のように管理するのではないだろうか。つまり、以下のような感じである。

- 時刻 $a$ に 2台入庫
- 時刻 $b$ に 1台出庫
- 時刻 $c$ に 3台入庫し、1台出庫
- 以下同様

つまり、駐車台数の変化のみに注目するという訳である。

この手法の利点は、車 1台につき「入庫情報」と「出庫情報」のみ書き換えればいいため、$O(1)$ で処理ができるという点にある。

ところが、実際に時刻 $T$ における台数を確認するためには、差分情報を元に累積和を計算する必要がある。従って、$O(N)$ の計算量が必要となるわけだ。

## いもす法の実装

いもす法については、抽象的な説明が難しいので、具体的な問題を解きつつ、実装を見ていくことにする。

---
**問題**  
駐車場に、ある日のべ $N$ 台の車の出入りがあった。それぞれの車の入庫時刻と出庫時刻が与えられる。駐車場に $P$台以上の車が駐車していた時間を求めなさい。

ただし、開始時点での駐車台数は $0$ 台とし、入庫と出庫に掛かる時間については無視するものとする。

**入力:**
$
N P\\
B_1 E_1\\
B_2 E_2\\
...\\
B_N E_N\\
$
$B_i, E_i$ は $i$台目の車の入庫時刻と出庫時刻である。また、入力は全て整数で行われる。

**制約**
$
1 \le N \le 10^6\\
1 \le B_i \le 10^6\\
1 \le E_i \le 10^6\\
$
各 $i$ $(1 \le i \le N)$ において $B_i \le E_i$

---

この問題を、いもす法を用いて順番に解いていくことを考えてみる。

まずは入力の受け取りから。

In [ ]:
N, P  = map(int, input().split())
car_dat = [list(map(int, input().split())) for _ in range(N)]

いもす法のために、 ```parking_diff``` というリストを作成しよう。```parking_diff[t]``` は、時刻 $t$ における駐車台数の変化とする。もちろん初期値は全て 0 である。長さは、制約一杯とっておくことにする。

In [ ]:
parking_diff = [0] * (10**6 + 1)

では、順に差分を記録していこう。各入庫時刻と出庫時刻に、駐車場内の車の台数は 1台増えたり減ったりする。

In [ ]:
for b, e in car_dat:
    parking_diff[b] += 1
    parking_diff[e] -= 1

差分の記録が終わったら、それを元に実際の台数を構築していく。つまりは累積和である。

In [ ]:
p_now = 0  # 現在の駐車台数
parking_num = [0] * (10**6 + 1)
for i, diff in enumerate(parking_diff):
    p_now += diff
    parking_num[i] = p_now

これで、各時刻における駐車台数が分かった。改めて、$P$ 台以上あった時間を求めていこう。

In [ ]:
answer = 0
for num in parking_num:
    if num >= P:
        answer += 1

以上をまとめると、

In [5]:
# 入力の受け取り
N, P  = map(int, input().split())
car_dat = [list(map(int, input().split())) for _ in range(N)]


# 差分の記録（いもす法）
parking_diff = [0] * (10**6 + 1)

for b, e in car_dat:
    parking_diff[b] += 1
    parking_diff[e] -= 1

# 実際の駐車台数を求める
p_now = 0  # 現在の駐車台数
parking_num = [0] * (10**6 + 1)
for i, diff in enumerate(parking_diff):
    p_now += diff
    parking_num[i] = p_now

# P 台以上の時間を求める
answer = 0
for num in parking_num:
    if num >= P:
        answer += 1

# 回答を出力
print(answer)

5 2
0 5
2 8
7 9
12 15
14 18
5


## いもす法＋座標圧縮

ところで、先程の問題では $1 \le B_i \le 10^6, 1 \le E_i \le 10^6$ という制約だった。つまり、時刻の上限は $10^6$ だったわけだが、この制約がもっと大きなケースがある。例えば、$1 \le B_i \le 10^9, 1 \le E_i \le 10^9$ のような感じである。

すると、最初のリスト構築と、最後の情報取り出しの時点で計算量が過大になってしまい、破綻することになってしまう。

こういうときには、座標圧縮と組み合わせてやると良い。

まずは、記録されている時刻を確認し、ソートしてやるところから始める。

In [ ]:
N, P = map(int, input().split())
car_dat = [list(map(int, input().split())) for _ in range(N)]

# 記録されている時刻を確認する
time_set = set()  # 重複を無くすため、set() を用いる

for b, e in car_dat:
    time_set.add(b)
    time_set.add(e)

# リストに変換してソートしておく
time_set = list(time_set)
time_set.sort()

これで、```time_set``` 内に使用されている全ての時刻が昇順で記録され、各時刻は ```time_set[i]``` と、インデックス ```i``` により呼び出すことができる様になる。次に、各時刻からインデックス、つまりその時刻が何番目の時刻なのかを求められるようにしておく。

In [11]:
time_to_index = dict()  # 辞書を使用する
for i, t in enumerate(time_set):
    time_to_index[t] = i

これらの情報を利用して、（記録が行われていない場所を無視して）記録が行われる場所のみを処理するようにする手法が、座標圧縮である。ただし、今回のように情報を全て先読みできる場合にのみ使用できる。

では、改めて圧縮後のインデックスを用いて、いもす法を行っていく。

In [12]:
parking_diff = [0] * len(time_set)  # 用いられている時刻の数だけ準備する

for b, e in car_dat:
    # 圧縮後の座標に変換し、差分を記録する
    b = time_to_index[b]
    e = time_to_index[e]
    
    parking_diff[b] += 1
    parking_diff[e] -= 1

# 同様に、圧縮後の座標を用いて車の台数を求める
p_num = 0
parking_num = [0] * len(time_set)
for i, d in enumerate(parking_diff):
    p_num += d
    parking_num[i] = p_num

これで、**各時刻に何台車がある状態に変化したのか**が分かるようになった。改めて答えを求めていく。

In [17]:
answer = 0

# 隣り合う二つの時刻ごとに確認していく
for i in range(len(parking_num) - 1):

    # ある時刻にP 台以上停まっていたら、
    # 次の時刻までの時間を答えに加算する
    if parking_num[i] >= P:

        start_time = time_set[i]
        end_time = time_set[i+1]
    
        answer += end_time - start_time

座標圧縮をしない場合と異なり、それなりに丁寧に処理を行う必要があるので、注意してほしい。

# 二次元累積和

累積和を二次元上で計算することもできる。次の例を見てもらいたい。

In [32]:
L = [
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10, 11, 12]
]

# 最初の行を [0, 0, 0, 0, 0] にしておく
csum_l = [[0] * (len(L[0]) + 1)]

for row in L:
    tmp = 0  # 対象の行のみの累積和
    csum_row = [0] * (len(L[0]) + 1)  # これから構築される二次元累積和の行

    for i, element in enumerate(row, start=1):
        tmp += element
        
        # 行の累積和に、手前の行の値を足しつつ保存していく
        csum_row[i] = tmp + csum_l[-1][i]
    
    # 行が完成したら、二次元累積和に追加
    csum_l.append(csum_row)

print(csum_l)

[[0, 0, 0, 0, 0], [0, 1, 3, 6, 10], [0, 6, 14, 24, 36], [0, 15, 33, 54, 78]]


少し見づらいので、行ごとに出力しておこう。

In [33]:
for row in csum_l:
    print(row)

[0, 0, 0, 0, 0]
[0, 1, 3, 6, 10]
[0, 6, 14, 24, 36]
[0, 15, 33, 54, 78]


確認できるように、左上 ```L[0][0]``` を始点として、右下 ```L[y][x]``` を含まない地点までの矩形範囲の値の合計が、```csum_l[y][x]``` で求められるようになる。

これを利用すると、この二次元リスト上の任意の地点間のあらゆる矩形範囲の合計を素早く求めることができるようになる。

その理屈は、下の図を見てもらいたい。

今、累積和によって $(0, 0)$ ～ 任意の点までの矩形範囲の合計を素早く求めることができる。つまり、$(A+B+C+D)$、$(A+B)$、$(A+C)$、$(A)$ については簡単に求められる。

すると、仮に$D$、つまり$(x1, y1)$ ～ $(x2, y2)$ の範囲を求めたければ、$(A+B+C+D) - ((A+B) + (A+C)) + A$ とすることで、$O(1)$ で求められるというわけだ。

気づいた方も多いと思うが、この計算はいわゆる包除原理に基づいている。つまり、任意の N次元に拡張可能となっている。ただ、競技プログラミングにおいては、ほぼ二次元までしか出ない。計算量の関係で、出すとしても三次元くらいが限界だろう。

先程の続きで実装して見るなら、以下のような感じだろう。

In [34]:
# 範囲 (x1, y1) ～ (x2, y2) の合計を求める（ただし x2, y2 は含まない半開放とする）
def get_rectangle_sum(x1, y1, x2, y2):
    # 上記の図と対応させるように、くどく書いてある
    A = csum_l[y1][x1]
    AB = csum_l[y1][x2]
    AC = csum_l[y2][x1]
    ABCD = csum_l[y2][x2]
    return ABCD - (AB + AC) + A

# テスト (2, 3, 6, 7)
print(get_rectangle_sum(1, 0, 3, 2))

18


# 二次元いもす法

累積和と同様、いもす法も二次元に適用でき、やはり矩形範囲に対する操作となる。

今 $(x1, y1)$ ～ $(x2, y2)$ に対して 1を加えるという操作をしたいとする。  
この場合の差分操作は、
- $(x1, y1)$ に 1 を足す
- $(x2, y1)$、$(x1, y2)$ から 1 を引く
- $(x2, y2)$ に 1 を足す  

というものになる。

実際の動作を確認してみよう。

In [10]:
# 上記操作を施したリスト
L = [
    [0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0,-1, 0],
    [0, 0, 0, 0, 0, 0],
    [0,-1, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0]
]

# 累積和を取ってみる。最初の 0 は入れないようにする。
csum_l = []
for i, row in enumerate(L):
    tmp = 0
    csum_row = []
    for j, element in enumerate(row):
        tmp += element
        
        # 二行目以降は前行の同じ位置の値を加えて保存
        if i == 0:
            csum_row.append(tmp)
        else:
            csum_row.append(tmp + csum_l[-1][j])

    csum_l.append(csum_row)

for row in csum_l:
    print(row)


[0, 0, 0, 0, 0, 0]
[0, 1, 1, 1, 0, 0]
[0, 1, 1, 1, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]


いい感じで矩形範囲に 1加算されているのが分かる。

この手法を用いることで、矩形範囲に対する操作クエリを $O(1)$ で行うことができる。

# 左右累積和

ここに含めるかどうか少し悩んだのだが、左右累積和と呼ばれる（ことがある）手法について、簡単に説明しておく。

今、あるリスト $L$ がある時に、二つのリスト ```left``` と ```right``` を作成する。

- left: ```L[:i]``` の累積和を ```left[i]``` で取得できるようなリスト
- right: ```L[i:]``` の累積和を ```right[i]``` で取得できるようなリスト

仮に整数のリスト L で作成してみると、以下のような感じになる。

In [28]:
L = [1, 2, 3, 4, 5]

left = [0] * (len(L)+1)
for i in range(len(L)):
    left[i+1] = left[i] + L[i]

right = [0] * (len(L)+1)
for i in range(len(L)-1, -1, -1):
    right[i] = right[i+1] + L[i]

print(left)
print(right)

[0, 1, 3, 6, 10, 15]
[15, 14, 12, 9, 5, 0]


これを利用すると、ある要素 ```L[i]``` を**除いた**合計を、簡単に求めることができる。

In [29]:
result = []
for i in range(len(L)):
    result.append(left[i] + right[i+1])
print(result)

[14, 13, 12, 11, 10]


もちろん、この手法は上記の例では全く役に立たない。なぜなら、総合計から各要素を引けば簡単に求まるからだ。

これがどのような時に役に立つのかというと、逆元が無い演算の時に役に立つ。

逆元とは、簡単に言うとある演算を打ち消す効果を持つもののことだ。単純な加算なら、ある数に対して符号を逆にしたものが逆元となる。  
上記の例で言うなら $1+2+3+4+5 = 15$ の中の $3$ の逆元は $-3$ なので、$15$ に $-3$ を加算すれば $1+2+4+5$ になる、という事実が、まさにこれにあたる。

逆元が無い演算……って具体的に何よ？という話になるが、簡単なところでは最小公倍数の計算などがそうだ。左右の累積で最小公倍数を持っておくことで、ある数列から特定の要素を除いた時の最小公倍数を素早く求めることができるようになる。

これを「累積和」と呼んでもいいのかどうかはまた別の問題として、この手法はたまに使うことがあるので、頭の片隅に引っ掛けておくといいだろう。